## Init

In [ ]:
import numpy as np
import scipy.constants as phy_const
import matplotlib.pyplot as plt
import pickle

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import os
import pandas as pd
import pickle
import glob
import sys
import configparser
import string

from post_treat_fcts import compute_phi, compute_E

listAlphabet = list(string.ascii_lowercase)



plt.style.use('/home/petronio/Documents/varie/style_fp.mplstyle')

plt.rcParams['axes.grid'] = True
plt.rcParams.update({'ytick.right': True, 'ytick.right': True})
plt.rcParams["font.family"] = "serif"

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout = self._original_stdout



In [ ]:
def read_input_and_set_globals(input_file):
    '''Reads the input file and sets the global variables
    TO BE COPIED AT THE BEGINNING OF THE MAIN FUNCTION, OTHERWISE THE GLOBAL VARIABLES WILL NOT BE SET'''
    global VG, M, m, R1, R2, A0, LENGTH, L0, alpha_B1, alpha_B2, mdot, Te_Cath, Rext, V0, Estar
    global NBPOINTS, SAVERATE, CFL, TIMEFINAL, TIMESCHEME, Delta_x, alpha_B

    config = configparser.ConfigParser()
    config.read(input_file)

    physicalParameters = config['Physical Parameters']

    VG       = float(physicalParameters['Gas velocity'])
    M        = float(physicalParameters['Ion Mass'])*phy_const.m_u
    m        = phy_const.m_e
    R1       = float(physicalParameters['Inner radius'])
    R2       = float(physicalParameters['Outer radius'])
    A0       = np.pi * (R2 ** 2 - R1 ** 2)
    LENGTH   = float(physicalParameters['Length of axis'])
    L0       = float(physicalParameters['Length of thruster'])
    alpha_B1 = float(physicalParameters['Anomalous transport alpha_B1'])
    alpha_B2 = float(physicalParameters['Anomalous transport alpha_B2'])
    mdot     = float(physicalParameters['Mass flow'])
    Te_Cath  = float(physicalParameters['Temperature Cathode'])
    Rext     = float(physicalParameters['Ballast resistor'])
    V0       = float(physicalParameters['Voltage'])
    Estar    = float(physicalParameters['Crossover energy'])
    # WALLCOLLS= bool(physicalParameters['Wall collisions'])               # Wall collisions
    Circuit  = bool(config.getboolean('Physical Parameters', 'Circuit', fallback=False)) # RLC Circuit


    NumericsConfig = config['Numerical Parameteres']

    NBPOINTS  = int(NumericsConfig['Number of points'])
    SAVERATE  = int(NumericsConfig['Save rate'])
    CFL       = float(NumericsConfig['CFL'])
    TIMEFINAL = float(NumericsConfig['Final time'])
    TIMESCHEME = NumericsConfig['Time integration']

    Delta_x  = LENGTH/NBPOINTS
    x_center = np.linspace(0, LENGTH, NBPOINTS)
    
    alpha_B = (np.ones(NBPOINTS) * alpha_B1)  # Anomalous transport coefficient inside the thruster
    alpha_B = np.where(x_center < L0, alpha_B, alpha_B2)  # Anomalous transport coefficient in the plume
    alpha_B_smooth = np.copy(alpha_B)

    # smooth between alpha_B1 and alpha_B2
    for index in range(10, NBPOINTS - 9):
        alpha_B_smooth[index] = np.mean(alpha_B[index-10:index+10])
    alpha_B = alpha_B_smooth
    
    # Magnetic field configuration
    MagneticFieldConfig = config['Magnetic field configuration']

    if MagneticFieldConfig['Type'] == 'Default':
        # print(MagneticFieldConfig['Type'] + ' Magnetic Field')

        Bmax     = float(MagneticFieldConfig['Max B-field'])                  # Max Mag field
        # LB       = float(MagneticFieldConfig['Length B-field'])               # Length for magnetic field
        # LBMax    = float(MagneticFieldConfig['Position maximum B-Field'])           # length of thruster (position of B_max)
        # saveBField = bool(MagneticFieldConfig['Save B-field'])

In [ ]:

##########################################################
#           POST-PROC PARAMETERS
##########################################################
Results     = "/home/petronio/Nextcloud/code/LPP1D/FLHET1D/Results/data_cp_2_2/"
Results     = "/home/petronio/Nextcloud/code/LPP1D/FLHET1D/Results/data_cp_2_2_noGradP/"

PLOT_VARS   = True
ResultConfig = Results+'/Configuration.cfg'

def get_files(ResultsData):
    files       = glob.glob(ResultsData + "/*.pkl")
    files.sort(key=os.path.getmtime)
    return files

root = "/home/petronio/Nextcloud/code/LPP1D/FLHET1D/Results/"
cases =  ["data_cp_2_2/", "data_cp_2_2_noGradP/"]

root = "/home/petronio/Nextcloud/code/LPP1D/FLHET1D/test_multiple_alpha_B_SPT100/alpha_CP15/"
cases = ["data_cp_GradP/", "data_cp_noGradP/"]

cases_labels = ["with GradP", "without GradP"]
print(root + cases[0])

Results = root + cases[1]
ResultConfig = Results+'/Configuration.cfg'
read_input_and_set_globals(ResultConfig)
print("Results: ", VG)


In [ ]:
# ##########################################################
# #           CONFIGURE PHYSICAL PARAMETERS
# ##########################################################
# configFile = ResultConfig
# print(ResultConfig)
# config = configparser.ConfigParser()
# config.read(configFile)

# physicalParameters = config['Physical Parameters']

# VG       = float(physicalParameters['Gas velocity'])                 # Gas velocity
# M        = float(physicalParameters['Ion Mass'])*phy_const.m_u       # Ion Mass
# m        = phy_const.m_e                                             # Electron mass
# R1       = float(physicalParameters['Inner radius'])                 # Inner radius of the thruster
# R2       = float(physicalParameters['Outer radius'])                 # Outer radius of the thruster
# A0       = np.pi * (R2 ** 2 - R1 ** 2)                               # Area of the thruster
# LENGTH   = float(physicalParameters['Length of axis'])               # length of Axis of the simulation
# L0       = float(physicalParameters['Length of thruster'])           # length of thruster (position of B_max)
# alpha_B1  = float(physicalParameters['Anomalous transport alpha_B1']) # Anomalous transport
# alpha_B2  = float(physicalParameters['Anomalous transport alpha_B2']) # Anomalous transport
# mdot     = float(physicalParameters['Mass flow'])                    # Mass flow rate of propellant
# Te_Cath  = float(physicalParameters['Temperature Cathode'])          # Electron temperature at the cathode
# Rext     = float(physicalParameters['Ballast resistor'])             # Resistor of the ballast
# V0       = float(physicalParameters['Voltage'])                      # Potential difference
# Estar    = float(physicalParameters['Crossover energy'])             # Crossover energy

# ##########################################################
# #           NUMERICAL PARAMETERS
# ##########################################################
# NumericsConfig = config['Numerical Parameteres']

# NBPOINTS  = int(NumericsConfig['Number of points'])             # Number of cells
# SAVERATE  = int(NumericsConfig['Save rate'])                    # Rate at which we store the data
# CFL       = float(NumericsConfig['CFL'])                        # Nondimensional size of the time step
# TIMEFINAL = float(NumericsConfig['Final time'])                 # Last time of simulation
# # Results   = NumericsConfig['Result dir']                        # Name of result directory
# TIMESCHEME = NumericsConfig['Time integration']                        # Name of result directory

# Delta_x  = LENGTH/NBPOINTS


In [ ]:
##########################################################
#           Make the plots
##########################################################

# plt.style.use('classic')
# plt.rcParams["font.family"] = 'serif'
# plt.rcParams["font.weight"] = 'normal'
# plt.rcParams['figure.facecolor'] = 'white'
# plt.rcParams["font.size"] = 15
# plt.rcParams["lines.linewidth"] = 2

ResultsFigs = Results+"Figs"
ResultsData = Results+"Data"
if not os.path.exists(ResultsFigs):
    os.makedirs(ResultsFigs)
print(ResultsData)

# open all the files in the directory and sort them to do the video in order
files       = glob.glob(ResultsData + "/*.pkl")
filesSorted = sorted(files, key = lambda x: os.path.getmtime(x), reverse=True)
files.sort(key=os.path.getmtime)
# print(files[:10])
# print(filesSorted[:10])

Current = np.zeros(np.shape(files)[0])
Voltage = np.zeros(np.shape(files)[0])
time    = np.zeros(np.shape(files)[0])

In [ ]:
# open all the files in the directory and sort them to do the video in order
files       = glob.glob(ResultsData + "/*.pkl")
filesSorted = sorted(files, key = lambda x: os.path.getmtime(x), reverse=True)
files.sort(key=os.path.getmtime)


Current = np.zeros(np.shape(files)[0])
Voltage = np.zeros(np.shape(files)[0])
time    = np.zeros(np.shape(files)[0])

In [ ]:

try :
    time,Current,Voltage = np.loadtxt(Results + "/acur_volt.dat")
    print("Loaded current and voltage values")
except:
    for i_save, file in enumerate(files):

        with open(file, 'rb') as f:
            [t, P, U, P_Inlet, P_Outlet, J, V, B, x_center] = pickle.load(f)

        # Save the current
        Current[i_save] = J
        Voltage[i_save] = V
        time[i_save]    = t
    np.savetxt(Results + "/cur_volt.dat", np.vstack((time,Current,Voltage)))

In [ ]:
%matplotlib widget
#####################################
#           Plot current
#####################################

f, ax = plt.subplots()

ax.plot(time/1e-3, Current, "o")
ax.set_xlabel(r'Time [ms]', fontsize=18)
ax.set_ylabel(r'Current [A]', fontsize=18)
ax_V=ax.twinx()
ax_V.plot(time/1e-3, Voltage,'r')
ax_V.set_ylabel(r'Voltage [V]', fontsize=18)
ax.grid(True)
# ax.set_ylim(0, np.amax(Current[100:]))
# ax.set_yscale('log')
plt.tight_layout()
plt.grid(False)
plt.xlim(0, np.amax(time/1e-3))

# plt.savefig(ResultsFigs+"/Current.png", bbox_inches='tight', transparent=True)

### Compare profiles 

In [ ]:
def get_files(ResultsData):
    files       = glob.glob(ResultsData + "/*.pkl")
    files.sort(key=os.path.getmtime)
    return files

root = "/home/petronio/Nextcloud/code/LPP1D/FLHET1D/Results/"
cases =  ["data_cp_2_2/", "data_cp_2_2_noGradP/"]

root = "/home/petronio/Nextcloud/code/LPP1D/FLHET1D/test_multiple_alpha_B_SPT100/alpha_CP15/"
cases = ["data_cp_GradP/", "data_cp_noGradP/"]

cases_labels = ["with GradP", "without GradP"]
print(root + cases[0])


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
for ind in [0,1]:
    Results = root + cases[ind]
    time,Current,Voltage = np.loadtxt(Results + "/cur_volt.dat")
    ax.plot(time/1e-3, Current, label = cases_labels[ind])
ax.set_xlabel(r'Time [ms]', fontsize=18)
ax.set_ylabel(r'Current [A]', fontsize=18)
ax.set_ylim(0, 10)
ax.set_xlim(0, 2)

plt.tight_layout()
plt.grid(False)

plt.legend()
plt.savefig("images_compare_gradP/Currents.png", bbox_inches='tight', transparent=True)

In [ ]:
fig = plt.figure(figsize=(10, 8))
gls = fig.add_gridspec(3, 1, hspace=0.1)
gls.update(left=0.15, right=0.75, wspace=0.05)
ax0 = fig.add_subplot(gls[0])
ax1 = fig.add_subplot(gls[1])

for ind in [0,1]:
    Results = root + cases[ind]
    ResultsData = Results + "Data"
    files = get_files(ResultsData)
    print(ResultsData, cases[ind], ind, len(files))
    read_input_and_set_globals(Results + "Configuration.cfg")

    # i_save = -1
    ng = np.zeros(NBPOINTS)
    ni = np.zeros(NBPOINTS)
    Te = np.zeros(NBPOINTS)

    for i_save, file in enumerate(files[-500:]):
        with open(file, 'rb') as f:
            [t, P, U, P_Inlet, P_Outlet, J, V, B, x_center] = pickle.load(f)
            ng += P[0,:]
            ni += P[1,:]
            Te += P[3,:]
            if i_save == 0:
                print(t*1e3)
    print(i_save+1, len(files[-500:]))
    ng /= (i_save+1)
    ni /= (i_save+1)
    Te /= (i_save+1)

    ax0.plot(x_center*100, ng, label = "$n_g$ " + cases_labels[ind])
    ax0.plot(x_center*100, ni, label = "$n_i$ " + cases_labels[ind])
    ax1.plot(x_center*100, Te, label = "$T_e$ " + cases_labels[ind])
    
ax0.set_ylabel(r'Density [m$^{-3}$]', fontsize=18)
ax1.set_ylabel(r'Temperature [eV]', fontsize=18)
ax1.set_xlabel(r'$x$ [cm]', fontsize=18)

ax0.set_title("average over last 100 time steps")

ax0.set_xlim(0, 5)
ax1.set_xlim(0, 5)
ax0.grid(False)
ax1.grid(False)
ax0.set_xticklabels([])

ax0.set_yscale('log')

ax0.legend(bbox_to_anchor=(1.001, 1), loc='upper left', frameon=False)
ax1.legend(bbox_to_anchor=(1.001, 1), loc='upper left', frameon=False)

fig.align_ylabels()

### Vi and Ve

In [ ]:
%load_ext autoreload
%autoreload 2

colors= ['tab:red', 'tab:green']
fig = plt.figure(figsize=(10, 8))
gls = fig.add_gridspec(3, 1, hspace=0.1)
gls.update(left=0.15, right=0.75, wspace=0.05)
ax0 = fig.add_subplot(gls[0])
ax1 = fig.add_subplot(gls[1])
ax2 = fig.add_subplot(gls[2])
ax2t = ax2.twinx()

for ind in [0,1]:
    Results = root + cases[ind]
    ResultsData = Results + "Data"
    read_input_and_set_globals(Results + "Configuration.cfg")
    files = get_files(ResultsData)
    print(ResultsData, cases[ind], ind, len(files))

    # i_save = -1
    ng = np.zeros(NBPOINTS)
    ni = np.zeros(NBPOINTS)
    vi = np.zeros(NBPOINTS)
    Te = np.zeros(NBPOINTS)
    ve = np.zeros(NBPOINTS)

    for i_save, file in enumerate(files[-500:]):
        with open(file, 'rb') as f:
            [t, P, U, P_Inlet, P_Outlet, J, V, B, x_center] = pickle.load(f)
            ng += P[0,:]
            ni += P[1,:]
            vi += P[2,:]
            Te += P[3,:]
            ve += P[4,:]
    print(i_save+1, len(files[-500:]))
    ng /= (i_save+1)
    ni /= (i_save+1)
    vi /= (i_save+1)
    Te /= (i_save+1)
    ve /= (i_save+1)

    ax0.plot(x_center*100, ng, color = colors[ind], label = "$n_g$ " + cases_labels[ind])
    ax0.plot(x_center*100, ni, color = colors[ind], label = "$n_i$ " + cases_labels[ind], ls = '--')
    ax1.plot(x_center*100, Te, color = colors[ind], label = "$T_e$ " + cases_labels[ind])
    ax2.plot(x_center*100, vi, color = colors[ind], label = "$v_i$ " + cases_labels[ind], ls = '--')
    ax2t.plot(x_center*100, ve, color = colors[ind], label = "$v_e$ " + cases_labels[ind])
    
ax0.set_ylabel(r'Density [m$^{-3}$]', fontsize=18)
ax1.set_ylabel(r'Temperature [eV]', fontsize=18)
ax2.set_ylabel(r'$v_i$ (dashed) [m/s]', fontsize=18)
ax2t.set_ylabel(r'$v_e$ [m/s]', fontsize=18)
ax2.set_xlabel(r'$x$ [cm]', fontsize=18)


ax0.set_xlim(0, 5)
ax1.set_xlim(0, 5)
ax2.set_xlim(0, 5)
ax0.grid(False)
ax1.grid(False)
ax2.grid(False)
ax2t.grid(False)
ax0.set_xticklabels([])
ax1.set_xticklabels([])

ax0.set_ylim(1e16, 1e20)
ax1.set_ylim(0, 15)
ax2.set_ylim(-5e3, 15e3)
ax2t.set_ylim(-1e5, 0)


ax0.set_yscale('log')

ax0.legend(bbox_to_anchor=(1.001, 1), loc='upper left', frameon=False)
ax1.legend(bbox_to_anchor=(1.001, 1), loc='upper left', frameon=False)

fig.align_ylabels()

plt.savefig("images_compare_gradP/Plasma_mean_profiles.png",dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
%matplotlib notebook
from ipywidgets import *

def slider(index, ind):
    Results = root + cases[ind]
    ResultsData = Results + "Data"
    read_input_and_set_globals(Results + "Configuration.cfg")
    files = get_files(ResultsData)
    file = files[index]

    with open(file, 'rb') as f:
        [t, P, U, P_Inlet, P_Outlet, J, V, B, x_center] = pickle.load(f)
        ng = P[0,:]
        ni = P[1,:]
        vi = P[2,:]
        Te = P[3,:]
        ve = P[4,:]
    return ng, ni, vi, Te, ve

colors= ['tab:red', 'tab:green']
fig = plt.figure(figsize=(10, 8))
gls = fig.add_gridspec(3, 1, hspace=0.1)
gls.update(left=0.15, right=0.75, wspace=0.05)
ax0 = fig.add_subplot(gls[0])
ax1 = fig.add_subplot(gls[1])
ax2 = fig.add_subplot(gls[2])
ax2t = ax2.twinx()

ng, ni, vi, Te, ve = slider(1000, 0)
ind = 0
line1, = ax0.plot(x_center*100, ng, color = colors[ind], label = "$n_g$ " + cases_labels[ind])
line2, = ax0.plot(x_center*100, ni, color = colors[ind], label = "$n_i$ " + cases_labels[ind], ls = '--')
line3, = ax1.plot(x_center*100, Te, color = colors[ind], label = "$T_e$ " + cases_labels[ind])
line4, = ax2.plot(x_center*100, vi, color = colors[ind], label = "$v_i$ " + cases_labels[ind], ls = '--')
line5, = ax2t.plot(x_center*100, ve, color = colors[ind], label = "$v_e$ " + cases_labels[ind])

ng, ni, vi, Te, ve = slider(1000, 1)
ind = 1
line11, = ax0.plot(x_center*100, ng, color = colors[ind], label = "$n_g$ " + cases_labels[ind])
line22, = ax0.plot(x_center*100, ni, color = colors[ind], label = "$n_i$ " + cases_labels[ind], ls = '--')
line33, = ax1.plot(x_center*100, Te, color = colors[ind], label = "$T_e$ " + cases_labels[ind])
line44, = ax2.plot(x_center*100, vi, color = colors[ind], label = "$v_i$ " + cases_labels[ind], ls = '--')
line55, = ax2t.plot(x_center*100, ve, color = colors[ind], label = "$v_e$ " + cases_labels[ind])


ax0.set_ylabel(r'Density [m$^{-3}$]', fontsize=18)
ax1.set_ylabel(r'Temperature [eV]', fontsize=18)
ax2.set_ylabel(r'$v_i$ (dashed) [m/s]', fontsize=18)
ax2t.set_ylabel(r'$v_e$ [m/s]', fontsize=18)
ax2.set_xlabel(r'$x$ [cm]', fontsize=18)


ax0.set_xlim(0, 5)
ax1.set_xlim(0, 5)
ax2.set_xlim(0, 5)
ax0.grid(False)
ax1.grid(False)
ax2.grid(False)
ax2t.grid(False)
ax0.set_xticklabels([])
ax1.set_xticklabels([])


ax0.set_yscale('log')

ax0.legend(bbox_to_anchor=(1.001, 1), loc='upper left', frameon=False)
ax1.legend(bbox_to_anchor=(1.001, 1), loc='upper left', frameon=False)

fig.align_ylabels()

def update(index):
    ng, ni, vi, Te, ve = slider(index, 0)
    line1.set_ydata(ng)
    line2.set_ydata(ni)
    line3.set_ydata(Te)
    line4.set_ydata(vi)
    line5.set_ydata(ve)

    ng, ni, vi, Te, ve = slider(index, 1)
    line11.set_ydata(ng)
    line22.set_ydata(ni)
    line33.set_ydata(Te)
    line44.set_ydata(vi)
    line55.set_ydata(ve)

    fig.canvas.draw_idle()
interact(update, index=(0, len(files)-1, 1))

## Calculate collision frequencies

In [ ]:
Kiz = 2.5e-13

colors_n = ['red', 'green', 'blue', 'orange', 'purple']
colorstab= ['tab:red', 'tab:green', 'tab:blue', 'tab:orange', 'tab:purple']
color_list = [colors_n, colorstab]

fig = plt.figure(figsize=(10, 6))
gls = fig.add_gridspec(2, 1, hspace=0.1)
gls.update(left=0.15, right=0.65, wspace=0.05)
ax0 = fig.add_subplot(gls[0])
ax1 = fig.add_subplot(gls[1])

for ind in [0,1]:
    colors = color_list[ind]
    print(colors)
    Results = root + cases[ind]
    ResultsData = Results + "Data"
    files = get_files(ResultsData)
    read_input_and_set_globals(Results + "Configuration.cfg")

    ng = np.zeros(NBPOINTS)
    ni = np.zeros(NBPOINTS)
    vi = np.zeros(NBPOINTS)
    Te = np.zeros(NBPOINTS)
    ve = np.zeros(NBPOINTS)

    for i_save, file in enumerate(files[-500:]):
        with open(file, 'rb') as f:
            [t, P, U, P_Inlet, P_Outlet, J, V, B, x_center] = pickle.load(f)
            ng += P[0,:]
            ni += P[1,:]
            vi += P[2,:]
            Te += P[3,:]
            ve += P[4,:]
    print(i_save+1, len(files[-500:]))
    ng /= (i_save+1)
    ni /= (i_save+1)
    vi /= (i_save+1)
    Te /= (i_save+1)
    ve /= (i_save+1)

    nu_iz = ng*Kiz # Ionization collision rate
    nu_an = alpha_B1*phy_const.e*B/m # Anomalous collision rate
    nu_an = np.where(x_center < L0, nu_an, alpha_B2*phy_const.e*B/m)
    print(alpha_B1, alpha_B2)
    nu_iw = ((4.0 / 3.0) * (1.0 / (R2 - R1)) * np.sqrt(phy_const.e * Te / M))  # Ion - wall collision rate
    sigma = 2.0 * Te / Estar  # SEE yield
    nu_ew = nu_iw / (1 - sigma)  # Electron - wall collision rate

    nu_tot = nu_iz + nu_an + nu_ew

    mobility = (phy_const.e / (m * nu_tot)) * (1. / (1 + (phy_const.e * B / (m * nu_tot)) ** 2))  # Effective mobility
    mobility_std = (phy_const.e / (m * (nu_iz + nu_ew))) * (1. / (1 + (phy_const.e * B / (m * (nu_iz + nu_ew))) ** 2))  # Effective mobility


    ax0.plot(x_center*100, nu_iz, color = colors[0], label = "$nu_{iz}$ " + cases_labels[ind], marker = 'v', markevery = (5*ind,10))
    ax0.plot(x_center*100, nu_an, color = colors[0+1], label = "$nu_{an}$ " + cases_labels[ind], marker = 'd', markevery = (5*ind,10))
    ax0.plot(x_center*100, nu_iw, color = colors[0+2], label = "$nu_{iw}$ " + cases_labels[ind], marker = 'X', markevery = (5*ind,10))
    ax0.plot(x_center*100, nu_ew, color = colors[0+3], label = "$nu_{ew}$ " + cases_labels[ind], marker = 'o', markevery = (5*ind,10))
    ax0.plot(x_center*100, nu_tot, color = colors[0+4], label = "$nu_{tot}$ " + cases_labels[ind], marker = 's', markevery = (5*ind,10))

    ax1.plot(x_center*100, mobility, color = colors[0], label = "mobility " + cases_labels[ind], marker = 's', markevery = (5*ind,10))
    ax1.plot(x_center*100, mobility_std, color = colors[0+1], label = "mobility std " + cases_labels[ind], marker = 'o', markevery = (5*ind,10))
    
ax0.set_ylabel(r'Coll. rate [s$^{-1}$]', fontsize=18)
ax1.set_ylabel(r'$\mu$ [m$^2$/Vs]', fontsize=18)
ax1.set_xlabel(r'$x$ [cm]', fontsize=18)

for ax in [ax0, ax1]:
    ax.set_xlim(0, 5)
    ax.grid(False)
    ax.set_yscale('log')
ax0.set_xticklabels([])

ax0.set_ylim(1e4, 1e8)
ax1.set_ylim(1e-2, 1e4)


ax0.set_yscale('log')
ax1.set_yscale('log')

ax0.legend(bbox_to_anchor=(1.001, 1), loc='upper left', frameon=False)
ax1.legend(bbox_to_anchor=(1.001, .5), loc='upper left', frameon=False)

fig.align_ylabels()

plt.savefig("images_compare_gradP/Plasma_coll_prof.png",dpi=300, bbox_inches='tight', transparent=True)

### Calculate potential and electric field

In [ ]:
Kiz = 2.5e-13

colors_n = ['red', 'green', 'blue', 'orange', 'purple']
colorstab= ['tab:red', 'tab:green', 'tab:blue', 'tab:orange', 'tab:purple']
markers = ['o', 's', 'v', 'd', 'X']
color_list = [colors_n, colorstab]

fig = plt.figure(figsize=(10, 6))
gls = fig.add_gridspec(2, 1, hspace=0.1)
gls.update(left=0.15, right=0.65, wspace=0.05)
ax0 = fig.add_subplot(gls[0])
ax1 = fig.add_subplot(gls[1])

for ind in [0,1]:
    colors = color_list[ind]
    Results = root + cases[ind]
    ResultsData = Results + "Data"
    files = get_files(ResultsData)
    configFile = Results + "Configuration.cfg"
    read_input_and_set_globals(configFile)

    Ex = np.zeros(NBPOINTS)
    phi = np.zeros(NBPOINTS)
    
    for i_save, file in enumerate(files[-1:]):
        with open(file, 'rb') as f:
            [t, P, U, P_Inlet, P_Outlet, J, V, B, x_center] = pickle.load(f)
            phi_id, Ex_id = compute_phi(P,J,B,configFile, V)
            phi += phi_id
            Ex += Ex_id
    Ex /= (i_save+1)
    phi /= (i_save+1)

    ax0.plot(x_center*100, phi, color = colors[2 + ind], label = "$\Phi$ " + cases_labels[ind], marker = markers[ind], markevery = (5*ind,10))
    ax1.plot(x_center*100, Ex/1e3, color = colors[0 + ind], label = "$E_x$ " + cases_labels[ind], marker = markers[2+ind], markevery = (5*ind,10))

    
ax0.set_ylabel(r'$\Phi$ [V]', fontsize=18)
ax1.set_ylabel(r'$E_x$ [kV/s]', fontsize=18)
ax1.set_xlabel(r'$x$ [cm]', fontsize=18)

for ax in [ax0, ax1]:
    ax.set_xlim(0, 5)
    ax.grid(False)
    ax.axhline(0, color='black', lw=1, ls='--')
ax0.set_xticklabels([])

ax0.set_ylim(0, 310)
ax1.set_ylim(-5, 30)


# ax0.set_yscale('log')
# ax1.set_yscale('log')

ax0.legend(bbox_to_anchor=(1.001, 1.), loc='upper left', frameon=False)
ax1.legend(bbox_to_anchor=(1.001, 1.), loc='upper left', frameon=False)

fig.align_ylabels()

plt.savefig("images_compare_gradP/Ex_phi_profiles.png",dpi=300, bbox_inches='tight', transparent=True)